In [1]:
file_path = "C:/Azure_kinect_project/test_MS1/test_MS2/m1_1.txt"

f = open(file_path, 'r')
lines = f.readlines()

output = []
frame = []
joint = 0
for line in lines:
    joint+=1
    line = line.strip()  # 줄 끝의 줄 바꿈 문자를 제거한다.
    line = line.split(":")
    line = line[1]
    
    line = line.strip(')')
    line = line.strip('(')
    line = line.split(",")

    for i in range(3):
        line[i] = float(line[i])
    frame.append(line)
    
    if joint%32 == 0:
        output.append(frame)
        joint = 0
        frame = []
f.close() 
print(output)

[[[47.696, -196.4, 1713.69], [59.917, -365.378, 1704.556], [71.818, -499.932, 1716.874], [88.211, -705.749, 1716.646], [117.48, -669.478, 1715.854], [242.029, -620.799, 1691.36], [289.66, -372.816, 1764.173], [246.382, -171.818, 1684.299], [288.561, -91.238, 1673.626], [267.98, 8.588, 1660.546], [268.279, -43.001, 1646.305], [53.743, -673.99, 1719.5], [-70.188, -667.446, 1734.998], [-195.547, -539.705, 1535.82], [-105.407, -742.818, 1558.458], [-73.206, -834.176, 1576.872], [-14.759, -915.312, 1591.127], [-65.835, -841.375, 1579.309], [133.624, -189.671, 1704.176], [138.508, 189.598, 1737.064], [129.686, 542.352, 1825.812], [131.983, 658.7, 1690.906], [-29.79, -202.468, 1722.268], [-32.598, 176.802, 1750.285], [-30.223, 536.664, 1825.671], [-40.358, 648.541, 1706.297], [93.571, -782.182, 1703.83], [-9.056, -819.985, 1597.097], [37.397, -845.967, 1596.378], [150.011, -821.048, 1631.48], [1.677, -855.237, 1629.701], [48.341, -861.826, 1745.047]], [[48.094, -196.125, 1713.921], [60.475, -

In [2]:
import numpy as np

In [3]:
#output[frame][joint][x,y,z]

# ∠
# the joints in test limb
angle_list = []
for i in range (len(output)):
    joint_1_xyz = np.array(output[i][4])  # a
    joint_2_xyz = np.array(output[i][7])  # b
    # the joints in plane
    joint_3_xyz = np.array(output[i][4])   # c
    joint_4_xyz = np.array(output[i][18])   # d

    # the vectors of above three joints.
    vector1 = joint_2_xyz - joint_1_xyz
    vector2 = joint_4_xyz - joint_3_xyz     # Fig_11_2 m09 Active straight raise
    # Vector2 = (1, 0, 0)   # Fig_12_2 m11 Trunk stability

    # the norm of vectors
    norm_vector1 = np.sqrt(vector1.dot(vector1))
    norm_vector2 = np.sqrt(vector2.dot(vector2))
    # the dot product of vectors
    dot_product = vector1.dot(vector2)
    # the radians of the angle
    angle_cos = dot_product / (norm_vector1 * norm_vector2)
    # convert to an angle value
    angle_deg = (np.arccos(angle_cos) * 180) / np.pi

    angle_list.append(angle_deg)
print(angle_list)
print(max(angle_list))

[12.75919147809354, 12.874726051238532, 12.874096925704476, 12.874096925704476, 13.779785448382171, 14.092476600438934, 15.373700273264411, 15.70200572433767, 16.303726996094117, 15.969434873648426, 16.04740620424934, 16.03287337398512, 15.802511678627505, 15.650194422826859, 15.489176317884898, 15.146078164329506, 15.143273665108431, 14.73213383417383, 14.232344213399315, 14.188072048128829, 14.187859944111459, 14.210676719467047, 14.223996683486115, 14.223996683486115, 14.223996683486115, 14.223996683486115, 14.25781114766587, 15.128604235718031, 15.399957165034458, 15.49934093581942, 15.780997637954266, 16.19968855037393, 16.27812920154396, 16.744861520838004, 18.1645756219773, 18.222531312916658, 19.22989910929562, 21.904785177307446, 23.2282913875001, 25.528447817932975, 27.306657159125013, 28.85018685313057, 31.2296523309412, 32.62581671905212, 34.74301608674003, 36.27736329823923, 38.74497418274823, 40.80194922697912, 42.768056347561135, 44.326270754334914, 46.68038822600625, 47